In [ ]:
%scala
import org.apache.spark.sql.SparkSession

In [ ]:
val spark = SparkSession
 .builder
 .appName("Departure Delays")
 .getOrCreate()

CARGAR ARCHIVO Y CONVERTIRLO EN DATAFRAME

In [ ]:
val csvFile= "departuredelays.csv"

val dframe = spark.read.format("csv")
 .option("inferSchema", "true")
 .option("header", "true")
 .load(csvFile)

CREAR VISTA TEMPORAL

In [ ]:
 dframe.createOrReplaceTempView("us_delay_flights_tbl")

CONSULTAS CON SPARK SQL

In [ ]:
spark.sql("""SELECT distance, origin, destination 
FROM us_delay_flights_tbl WHERE distance > 1000 
ORDER BY distance DESC""").show(10)

In [ ]:
spark.sql("""SELECT date, delay, origin, destination 
FROM us_delay_flights_tbl 
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD' 
ORDER by delay DESC""").show(10)

HACER EN DATABRICKS

In [ ]:
spark.sql("""CREATE TABLE us_delay_flights_tbl(date STRING, delay INT, 
 distance INT, origin STRING, destination STRING) 
 USING csv OPTIONS (PATH 
 'departuredelays.csv')""")

In [ ]:
spark.sql("""SELECT delay, origin, destination,
 CASE
 WHEN delay > 360 THEN 'Very Long Delays'
 WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
 WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
 WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
 WHEN delay = 0 THEN 'No Delays'
 ELSE 'Early'
 END AS Flight_Delays
 FROM us_delay_flights_tbl
 ORDER BY origin, delay DESC""").show(10)

In [ ]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)

In [ ]:
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

CREAR DATAFRAME CON LOS ARCHIVOS DE DIFERENTES FORMATOS

In [ ]:
val file = """firecalls_parquet/part-00000-5c94c46a-88ec-4efd-b88c-f6f2d3540b13-c000.snappy.parquet"""

val df_parquet = spark.read.format("parquet").load(file)

val df_csv = spark.read.format("csv")
 .option("inferSchema", "true")
 .option("header", "true")
 .option("mode", "PERMISSIVE")
 .load("work/firecalls_csv/part-00000-a31139a7-b7d7-4c79-89d1-ecd5f6d5adbe-c000.csv*")

val df4_json = spark.read.format("json")
 .load("work/firecalls_json/part-00000-59c45c44-7990-4e9f-8ed2-78dc0f165b61-c000.json*")

In [ ]:
val location = dframe.write.format("json").mode("overwrite").save("work/Prueba Dataframes")